<a href="https://colab.research.google.com/github/Gilbert9172/CSAI_assignment/blob/main/%5B%EC%A0%95%EA%B8%B8%EC%A4%80%5Dn222.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) 


In [1]:
pip install --upgrade category_encoders

     |████████████████████████████████| 81kB 2.8MB/s 


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

target = 'vacc_h1n1_f'

train_data = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'), 
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')

In [3]:
# opinion_h1n1_vacc_effective; 벡신접종 후 영향에  대한의견이 순서형 범주에 해당
train_data['opinion_h1n1_vacc_effective'].value_counts()

Somewhat Effective      14729
Very Effective           9052
Dont Know                6033
Not Very Effective       2312
Not At All Effective     1111
Refused                   138
Name: opinion_h1n1_vacc_effective, dtype: int64

In [4]:
# OrdinalEncoder 인코딩
# from category_encoders import OrdinalEncoder 

# Ord_enc = OrdinalEncoder(handle_missing="value")

# enc = {"Dont Know":0, "Refused": 1, "Not At All Effective": 2, "Not Very Effective": 3, "Somewhat Effective ":4,"Very Effective":5}

# train_data['opinion_h1n1_vacc_effective'] = train_data.opinion_h1n1_vacc_effective.map(enc)

# # NaN값 = 0 : Dont Know
# train_data['opinion_h1n1_vacc_effective'] = train_data['opinion_h1n1_vacc_effective'].fillna(0)

In [5]:
train, val = train_test_split(train_data, train_size=0.80, test_size=0.20, 
                              stratify=train_data[target], random_state=2)
def engineer(df):
    selected_cols = df.select_dtypes(include=['number', 'object'])
    labels = selected_cols.nunique() 
    selected_features = labels[labels <= 30].index.tolist() 
    df = df[selected_features]

    df.drop(['employment_industry','employment_occupation'], axis = 1, inplace=True)    
    return df

In [6]:
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = test[features]

# 2)

In [7]:
# 연습 (문제점 : 너무 느리다. 나무 성장과정 보는 수준)
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.impute import SimpleImputer  
# from sklearn.pipeline import make_pipeline
# from sklearn.model_selection import GridSearchCV

# pipe_ord = make_pipeline(
#     OrdinalEncoder(),
#     SimpleImputer(strategy = 'median'),
#     RandomForestClassifier(random_state = 0, 
#                            n_jobs = -1, 
#                            oob_score = True)
# )

In [8]:
# sorted(pipe_ord.get_params().keys())

In [9]:
# param_grid = {
#     'randomforestclassifier__n_estimators': [100,200],
#     'randomforestclassifier__max_depth': [6,8,10,12],
#     'randomforestclassifier__min_samples_leaf': [2,6,10],
#     'randomforestclassifier__min_samples_split' : [2,4,6,8]
# }

In [10]:
# rf_grid = GridSearchCV(pipe_ord,param_grid=param_grid,scoring="accuracy",
#                        n_jobs = -1, verbose=1)

In [11]:
# rf_grid.fit(X_train,y_train)



```
Fitting 5 folds for each of 96 candidates, totalling 480 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 23.8min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 26.0min finished
GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('ordinalencoder',
                                        OrdinalEncoder(cols=None,
                                                       drop_invariant=False,
                                                       handle_missing='value',
                                                       handle_unknown='value',
                                                       mapping=None,
                                                       return_df=True,
                                                       verbose=0)),
                                       ('simpleimputer',
                                        SimpleImputer(add_indicator=False,
                                                      copy=True,
                                                      fill_value=None,
                                                      missing_values=nan,
                                                      strategy='median',
                                                      verbose=0))...
                                                               warm_start=False))],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'randomforestclassifier__max_depth': [6, 8, 10, 12],
                         'randomforestclassifier__min_samples_leaf': [2, 6, 10],
                         'randomforestclassifier__min_samples_split': [2, 4, 6,
                                                                       8],
                         'randomforestclassifier__n_estimators': [100, 200]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=1)
```



In [12]:
# rf_grid.best_params_



```
{'randomforestclassifier__max_depth': 12,
 'randomforestclassifier__min_samples_leaf': 6,
 'randomforestclassifier__min_samples_split': 2,
 'randomforestclassifier__n_estimators': 100}
```



In [ ]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer  
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score
from category_encoders import OrdinalEncoder 

pipe_ord = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy = 'mean'),
    RandomForestClassifier(n_estimators = 100, 
                           max_depth=12,
                           class_weight='balanced',
                           min_samples_split = 2,
                           min_samples_leaf = 6,
                           random_state = 1, 
                           n_jobs = -1, 
                           oob_score = True)
)

pipe_ord.fit(X_train,y_train)

y_v_pred = pipe_ord.predict(X_val)
print('학습정확도', pipe_ord.score(X_train,y_train))
print('검증정확도', pipe_ord.score(X_val,y_val))
print('F1_Score', f1_score(y_val, y_v_pred))

# 학습정확도 0.8337040002372268
# 검증정확도 0.800260941762543
# F1_Score 0.6308636562911004

In [14]:
# 연습 : ExtraTreesClassifier ()
# from sklearn.ensemble import ExtraTreesClassifier

# pipe_ord = make_pipeline(
#     OrdinalEncoder(),
#     SimpleImputer(strategy = 'mean'),
#     ExtraTreesClassifier(n_estimators = 100, 
#                            bootstrap=True,
#                            max_depth=12,
#                            class_weight='balanced',
#                            min_samples_split = 2,
#                            min_samples_leaf = 6,
#                            random_state = 1, 
#                            n_jobs = -1, 
#                            oob_score = True)
# )

# pipe_ord.fit(X_train,y_train)

# y_v_pred = pipe_ord.predict(X_val)
# print('학습정확도', pipe_ord.score(X_train,y_train))
# print('검증정확도', pipe_ord.score(X_val,y_val))
# print('F1_Score', f1_score(y_val, y_v_pred))


#3)


- Target encoding

In [15]:
RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer  
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score
from category_encoders import TargetEncoder


pipe_ord = make_pipeline(
    TargetEncoder(),
    SimpleImputer(strategy = 'mean'),
    RandomForestClassifier(n_estimators = 100, 
                           max_depth=12,
                           class_weight='balanced',
                           min_samples_split = 2,
                           min_samples_leaf = 5,
                           random_state = 1, 
                           n_jobs = -1, 
                           oob_score = True)
)
pipe_ord.fit(X_train,y_train)

y_v_pred = pipe_ord.predict(X_val)
print('학습정확도', pipe_ord.score(X_train,y_train))
print('검증정확도', pipe_ord.score(X_val,y_val))
print('F1_Score', f1_score(y_val, y_v_pred))

# 학습정확도 0.8383892299024405
# 검증정확도 0.8070217056102479
# F1_Score 0.6407595495694414

In [16]:
y_predict = pipe_ord.predict(X_test)

sample_submission = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/submission.csv')

sample_submission['vacc_h1n1_f'] = y_predict

sample_submission = sample_submission.set_index('Id')

sample_submission

In [17]:
# sample_submission.to_csv('submission_g.csv')

[Target encoding (번역)](https://brendanhasz.github.io/2019/03/04/target-encoding#target-encoding)

```
원핫인코딩의 문제점은, 학습데이터의 차원을 아주 많이 증가시킨다는 점입니다.

이 문제는 종종 모델의 성능을 저하 시키고, 차원의 저주에 빠지게 합니다. 

반면에 타겟 인코딩은 우리가 좋은 정보를 얻을 수 있게 유용한 카테고리를 유지시켜 줍니다.

(이 부분에서 원핫인코딩과 비슷하지만, 라벨 인코딩과는 사뭇 다른 느낌입니다. )

동시에 타겟인코딩은 인코딩 전의 차원을 그대로 유지 시켜줍니다.

(이 부분에서는 라벨인코딩과 비슷하고, 원핫인코딩과 차이가 납니다.)

각 피쳐의 데이터에 대해서 타겟 인코딩을 하기 위해서는, 각 카테고리를 해당 카테고리 샘플의 평균으로 대체해줍니다.
```





 

